# Observational Studies

To draw causal conclusions on the impact of our features on movie success, we have to perform the standard operations used in observational studies, such as propensity matching and regression.

## Packages

In [47]:
# Global packages
import pandas as pd
import numpy as np
# Statistical package
import statsmodels.api as sm
# Matching package
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
# Custom helpers
import feature_and_regression as feat_and_reg
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [4]:
raw_regression_df = feat_and_reg.get_raw_regression_df()

In [195]:
processed_df, target, binary_target, num_votes = feat_and_reg.format_regression_df(raw_regression_df,[2010])
processed_df.head()

,North America and Australia,Western Europe,Asia,Africa and Middle-East,Eastern Europe and Russia,Central and South America,gender_ratio,has_famous_actor,action,adventure,...,horror,animation,children,adult,fantasy,genre,title_length,combinned_movie_num,num_directors,combinned_movie_success
movie_id,,,,,,,,,,,,,,,,,,,,,
27463222,1.0,0.0,0.0,0.0,0.0,0.0,-0.285714,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.002276,5.0,1.0,1
27504492,0.0,0.0,1.0,0.0,0.0,0.0,-0.200000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.358348,2.0,1.0,0
35785312,0.0,1.0,0.0,0.0,0.0,0.0,-0.166667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.038660,1.0,1.0,0
20506905,0.0,1.0,0.0,0.0,0.0,0.0,-0.764706,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.358348,1.0,1.0,0
24021571,1.0,0.0,0.0,0.0,0.0,0.0,-0.466667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.358348,2.0,1.0,0


## Propensity Score

In [196]:
psm = PsmPy(processed_df.reset_index(), treatment='has_famous_actor', indx='movie_id', exclude = [])

In [197]:
# Compute propensity scores
psm.logistic_ps(balance = True)

In [198]:
# Creates matching
psm.knn_matched(matcher='propensity_score', replacement=False, caliper=None)
matched_ids = set(psm.matched_ids["movie_id"]).union(psm.matched_ids["matched_ID"])

In [200]:
# Regression
features = feat_and_reg.forward_selection(processed_df, target)
model = sm.OLS(target, sm.add_constant(processed_df["has_famous_actor"])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         average_rating   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.318
Date:                Fri, 16 Dec 2022   Prob (F-statistic):              0.251
Time:                        18:01:30   Log-Likelihood:                -1421.0
No. Observations:                 991   AIC:                             2846.
Df Residuals:                     989   BIC:                             2856.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                6.1901      0.036  

## Observationnal study pipeline

In [188]:
def binarize_treatment(processed_dataframe: pd.DataFrame,
                       treatment: str, threshold: float, method="greater"):
    """
    
    :param processed_dataframe: Pandas DataFrame containing the data for regression.
    :param treatment: Covariate used as treament (should be binary).
    :param threshold: Threshold value for the binarization.
    :param method: Method for binarization with respect to the threshold.
    """
    if method == "greater":
        f = lambda l: l > threshold
    elif method == "smaller":
        f = lambda l: l < threshold
    elif method == "equal":
        f = lambda l: l == threshold
    elif method == "unequal":
        f = lambda l: l != threshold
    else:
        ValueError("Please provide a method in [greater,smaller,equal,unequal].")
    processed_dataframe[treatment] = processed_dataframe[treatment].apply(f)
    processed_dataframe[treatment] = processed_dataframe[treatment].replace({True: 1, False: 0})
    
def temporal_obs_study(raw_regression_df: pd.DataFrame, treatment: str,
                       decades: list, binarize=False, threshold=0,
                       method="greater",alpha=0.05) -> tuple:
    """
    Perform observational study accros the decades provided.
    
    :param raw_regression_df: Pandas DataFrame with raw data for regression.
    :param treatment: Covariate used as treament (should be binary).
    :param decades: List of decades for which the function will perform the analysis.
    :param binarize: Indicator if a binirazation of the treatment is necessary.
    :param threshold: Threshold value for the binarization.
    :param method: Method for binarization with respect to the threshold.
    :param alpha: Significance level, default 0.05.
    
    :return: Tuple with dictionnary with the model for each decade, and a summary dataframe.  
    
    """
    decades_models = dict()
    decade_significance = dict()
    for decade in decades:
        processed_df, target, binary_target, num_votes = feat_and_reg.format_regression_df(
                                                                raw_regression_df,[decade])
        # If necessary, binarize treatment
        if binarize:
            binarize_treatment(processed_df, treatment,
                               threshold, method=method)
        if (processed_df[treatment].sum() == 0 or 
            processed_df[treatment].sum() == len(processed_df)):
            print("Sample have all undergoe same treatment in data.")
            decade_significance[decade] = (False, None)
            continue
        # Compute propensity scores
        psm = PsmPy(processed_df.reset_index(), treatment=treatment, indx='movie_id')
        psm.logistic_ps(balance = True)
        # Creates matching
        psm.knn_matched(matcher='propensity_score', replacement=False, caliper=None)
        matched_ids = set(psm.matched_ids["movie_id"]).union(psm.matched_ids["matched_ID"])
        # Regression
        features = feat_and_reg.forward_selection(processed_df, target)
        model = sm.OLS(target, sm.add_constant(processed_df[features])).fit()
        # Update results
        decades_models[decade] = model
        if treatment in model.pvalues:
            decade_significance[decade] = (model.pvalues[treatment] < alpha,
                                          model.params[treatment])
        else:
            decade_significance[decade] = (False, None)
    decade_results_df = pd.DataFrame(decade_significance.values(),index=decade_significance.keys(),
                                    columns=["treatment_significant","coeff"]).sort_index()
    return decades_models, decade_results_df

In [193]:
model_list, decade_df = temporal_obs_study(
    raw_regression_df,"gender_ratio",[1970,1980,1990,2000,2010],binarize=True,
    threshold=0,method="unequal",alpha=0.05)

1970
1980
1990
2000
2010


In [194]:
decade_df

,treatment_significant,coeff
1970,False,NaN
1980,False,NaN
1990,False,NaN
2000,True,0.132241
2010,True,-0.199547
